In [ ]:
# Install the packages\
%pip install requests openai python-dotenv praw

In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Now you can access the variables using os.getenv
gpt4_azure_openai_endpoint = os.getenv("GPT4_AZURE_OPENAI_ENDPOINT")
gpt4_azure_openai_key = os.getenv("GPT4_AZURE_OPENAI_KEY")
gpt4_azure_openai_api_version = os.getenv("GPT4_AZURE_OPENAI_API_VERSION")
gpt4_deployment_name = os.getenv("GPT4_DEPLOYMENT_NAME")


In [ ]:
# Assistant stuff

from openai import AzureOpenAI
from typing import Optional
import json
import time

from pathlib import Path


def create_message(
    client: AzureOpenAI,
    thread_id: str,
    role: str = "",
    content: str = "",
    file_ids: Optional[list] = None,
    metadata: Optional[dict] = None,
    message_id: Optional[str] = None,
) -> any:
    """
    Create a message in a thread using the client.

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param role: Message role (user or assistant)
    @param content: Message content
    @param file_ids: Message file IDs
    @param metadata: Message metadata
    @param message_id: Message ID
    @return: Message object

    """
    if metadata is None:
        metadata = {}
    if file_ids is None:
        file_ids = []

    if client is None:
        print("Client parameter is required.")
        return None

    if thread_id is None:
        print("Thread ID is required.")
        return None

    try:
        if message_id is not None:
            return client.beta.threads.messages.retrieve(thread_id=thread_id, message_id=message_id)

        if file_ids is not None and len(file_ids) > 0 and metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids, metadata=metadata
            )

        if file_ids is not None and len(file_ids) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids
            )

        if metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, metadata=metadata
            )

        return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content)

    except Exception as e:
        print(e)
        return None

def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param run_id: Run ID
    @param assistant_id: Assistant ID
    @param verbose: Print verbose output
    @param max_steps: Maximum number of steps to poll
    @param wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if verbose:
                                print(f"Function call: {call.function.name}")
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)

def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param verbose: Print verbose output
    @param out_dir: Output directory to save images
    @return: Messages object

    """

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User query", "assistant": "Assistant response"}

        prev_role = None

        if verbose:
            print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == "assistant" and md.role == "user" and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

# Functions

In [ ]:
# init tool array

tools = []
functions_map = {}

In [ ]:
#Reddit

import os
import json
from dotenv import load_dotenv
import praw

# Load the .env file
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
user_agent = os.getenv("USER_AGENT")

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

def subbmission_to_json(submission):
    return {
        "title": submission.title,
        "score": submission.score,
        "id": submission.id,
        "url": submission.url,
        "num_comments": submission.num_comments,
        "created": submission.created,
        "selftext": submission.selftext
    }

def get_reddit_posts(subreddit, limit):
    hot_posts = reddit.subreddit(subreddit).hot(limit=limit)
     
    return json.dumps([subbmission_to_json(post) for post in hot_posts])

# print(get_reddit_posts("python", 5))

def subreddit_search(keyword):
    subs = reddit.subreddits.search_by_name(keyword, exact=False)
    return json.dumps([sub.display_name for sub in subs])

print(subreddit_search("python"))

#FUDNCTION DESCRIPTION
top_subreddit_posts ={
      
    "type": "function",
    "function": {
        "name": "get_reddit_posts",
        "description": "Gets the top posts from a specified subreddit.",
        "parameters": {
            "type": "object",
            "properties": {
                "subreddit": {
                    "type": "string",
                    "description": "The subreddit to get posts from",
                },
                "limit": {
                    "type": "integer",
                    "description": "The number of top posts to retrieve",
                }
            },
            "required": ["subreddit", "limit"],
        },
    },
}

tools.append(top_subreddit_posts)
functions_map["get_reddit_posts"] = get_reddit_posts

In [ ]:
# Write file

def write_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return filename

# Example usage:
write_to_file('example.txt', 'Hello, world!')

write_to_file_function = {
    "type": "function",
    "function": {
        "name": "write_to_file",
        "description": "Writes a given string into a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "filename": {
                    "type": "string",
                    "description": "The name of the file to write to",
                },
                "content": {
                    "type": "string",
                    "description": "The string to write into the file",
                }
            },
            "required": ["filename", "content"],
        },
    },
}

tools.append(write_to_file_function)
functions_map["write_to_file"] = write_to_file

# Define Assistant

In [ ]:

def get_message(sub_reddit, limit=5):
    return f"Please write a newsletter about the {limit} most importnant topics from the {sub_reddit} subreddit? Please save the newsletter in a file {sub_reddit}_newsletter.md"

def get_assistant(sub_reddit, limit=5):
    return  {
        'name' : "reddit_assistant",
        'instructions' : """You are an assistant designed to help users getting an overview what happens on a subreddit.

You have access to the subreddit api and create a newsletter like summary the top posts of a subreddit for the user.
""",
        'message': {
            'role': 'user',
            'content': get_message(sub_reddit, limit)
        }
    }


In [ ]:


def get_message(sub_reddit, limit=5):
    return f"Please write a newsletter about the {limit} most importnant topics from the {sub_reddit} subreddit? Please save the newsletter in a file {sub_reddit}_newsletter.md"

def get_assistant(sub_reddit, limit=5):
    return  {
        'name' : "reddit_assistant",
        'instructions' : """You are an assistant designed to help users getting an overview what happens on a subreddit.

You have access to the subreddit api and create a newsletter like summary the top posts of a subreddit for the user.
""",
        'message': {
            'role': 'user',
            'content': get_message(sub_reddit, limit)
        }
    }


In [ ]:
def call_assistant(assistant_def, verbose_output):
    client = AzureOpenAI(api_key=gpt4_azure_openai_key, api_version=gpt4_azure_openai_api_version, azure_endpoint=gpt4_azure_openai_endpoint)
    assistant = client.beta.assistants.create(
    name=assistant_def['name'], description="", instructions=assistant_def['instructions'], tools=tools, model=gpt4_deployment_name)
    thread = client.beta.threads.create()
    create_message(client, thread.id, assistant_def['message']["role"], assistant_def['message']["content"])
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id, instructions=assistant_def['instructions'])
    poll_run_till_completion(client=client, thread_id=thread.id, run_id=run.id, available_functions=functions_map, verbose=verbose_output, max_steps=50)
    return retrieve_and_print_messages(client=client, thread_id=thread.id, verbose=verbose_output)

# Execution


In [ ]:
assistant_def = get_assistant("german", 10)

verbose_output = True

call_assistant(assistant_def, verbose_output)